# Storage

These are very important opcodes because they manipulate storage, which is very expensive.

#### Storage Load

Load a `word` from storage given by a `key` and put it on the stack.

In [ ]:
pub fn sload(evm: &mut EVM) {
    let key = evm.stack.pop().value;
    let (warm, value) = evm.storage.load(key);
    evm.stack.push(value);

    evm.gas_dec(2100); // 100 if warm
    evm.pc += 1;
}

#### Storage Store

Get the `key` and the `word` that will be stored from the stack and store in storage.

In [ ]:
pub fn sstore(evm: &mut EVM) {
    let key = evm.stack.pop();
    let value = evm.stack.pop();
    let (warm, old_value) = evm.storage.store(key, value);

    let mut base_dynamic_gas = 0;

    if value != old_value {
        if old_value == 0 {
            base_dynamic_gas = 20000;
        } else {
            base_dynamic_gas = 2900;
        }
    }

    let access_cost = if warm { 100 } else { 2100 };
    evm.gas_dec(base_dynamic_gas + access_cost);

    evm.pc += 1;

    // TODO: handle refunds
}